In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import r2_score, make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.grid_search import GridSearchCV
from sklearn import linear_model

%matplotlib inline

In [2]:
infile_old = '../data/BEA-RegionalIncomeByIndustry/CA5_1969_2000_MSA.csv'
infile = '../data/BEA-RegionalIncomeByIndustry/CA5N_2001_2013_MSA.csv'

df_old = pd.read_csv(infile_old)
df = pd.read_csv(infile)

df_old


/usr/local/lib/python2.7/dist-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


,GeoFIPS,GeoName,Region,Table,LineCode,IndustryClassification,Description,1969,1970,1971,...,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000
0,10180,"Abilene, TX (Metropolitan Statistical Area)",NaN,CA5,10,...,Personal income,395971,443561,474234,...,2376433,2538036,2663665,2722081,2918059,3109606,3329032,3490073,3639311,3741391
1,10180,"Abilene, TX (Metropolitan Statistical Area)",NaN,CA5,20,...,Population (persons) 2/,125138,122505,124841,...,147400,149917,152909,153779,156097,156351,157405,158264,159755,160288
2,10180,"Abilene, TX (Metropolitan Statistical Area)",NaN,CA5,30,...,Per capita personal income (dollars),3164,3621,3799,...,16122,16930,17420,17701,18694,19889,21149,22052,22781,23342
3,10180,"Abilene, TX (Metropolitan Statistical Area)",NaN,CA5,35,...,Earnings by place of work,314729,345979,368356,...,1640392,1759145,1834506,1895221,1995628,2131048,2326369,2432863,2579609,2598895
4,10180,"Abilene, TX (Metropolitan Statistical Area)",NaN,CA5,36,...,less: Contributions for government social in...,18303,19934,22303,...,168262,176727,184213,193361,203789,214107,227052,232301,240171,244306
5,10180,"Abilene, TX (Metropolitan Statistical Area)",NaN,CA5,37,...,Employee and self-employed contributions f...,9557,10240,11245,...,87427,91460,95847,101582,108123,113405,120785,123957,128823,129272
6,10180,"Abilene, TX (Metropolitan Statistical Area)",NaN,CA5,38,...,Employer contributions for government soci...,8746,9694,11058,...,80835,85267,88366,91779,95666,100702,106267,108344,111348,115034
7,10180,"Abilene, TX (Metropolitan Statistical Area)",NaN,CA5,42,...,plus: Adjustment for residence 4/,1495,1863,2003,...,5066,6192,6245,4630,3561,3165,4426,6403,7632,10171
8,10180,"Abilene, TX (Metropolitan Statistical Area)",NaN,CA5,45,...,equals: Net earnings by place of residence,297921,327908,348056,...,1477196,1588610,1656538,1706490,1795400,1920106,2103743,2206965,2347070,2364760
9,10180,"Abilene, TX (Metropolitan Statistical Area)",NaN,CA5,46,...,"plus: Dividends, interest, and rent 5/",61986,73938,78915,...,542650,542405,579228,557362,624395,657686,668241,720342,709218,772092


In [40]:
# clean the data columns
import re

data_df = df_old[[str(x) for x in range(1969,2001)]]


def clean_data_entry(x):
    if isinstance(x, basestring):
        m = re.match("\d+",x)
        if m:
            return float(m.group())
        else:
            return np.nan
    else:
        return x

clean_df = df_old.copy()
for yr in range(1969,2001):
    clean_df[str(yr)] = clean_df[str(yr)].apply(clean_data_entry)

print clean_df.head(3)

  GeoFIPS                                      GeoName  Region Table  \
0   10180  Abilene, TX (Metropolitan Statistical Area)     NaN   CA5   
1   10180  Abilene, TX (Metropolitan Statistical Area)     NaN   CA5   
2   10180  Abilene, TX (Metropolitan Statistical Area)     NaN   CA5   

   LineCode IndustryClassification                           Description  \
0        10                    ...                       Personal income   
1        20                    ...               Population (persons) 2/   
2        30                    ...  Per capita personal income (dollars)   

     1969    1970    1971   ...        1991     1992     1993     1994  \
0  395971  443561  474234   ...     2376433  2538036  2663665  2722081   
1  125138  122505  124841   ...      147400   149917   152909   153779   
2    3164    3621    3799   ...       16122    16930    17420    17701   

      1995     1996     1997     1998     1999     2000  
0  2918059  3109606  3329032  3490073  3639311  374

In [42]:
for geo in clean_df['GeoFIPS'].unique():
    numna = clean_df[clean_df['GeoFIPS']==geo][[str(x) for x in range(1969,2001)]].isnull().sum().sum()
    print numna, geo

735 10180
474 10420
1082 10500
545 10540
940 10580
620 10740
750 10780
828 10900
593 11020
841 11100
787 11180
688 11260
628 11460
586 11500
713 11540
939 11700
1035 12020
1097 12060
673 12100
643 12220
1313 12260
668 12420
476 12540
813 12580
590 12620
673 12700
950 12940
705 12980
661 13020
805 13140
771 13220
500 13380
598 13460
630 13740
845 13780
770 13820
926 13900
1264 13980
934 14010
835 14020
964 14100
831 14260
536 14460
498 14500
1017 14540
556 14740
377 14860
598 15180
871 15260
404 15380
639 15500
687 15540
691 15680
510 15940
454 15980
915 16020
1064 16060
782 16180
538 16220
686 16300
689 16540
853 16580
627 16620
841 16700
979 16740
1148 16820
694 16860
685 16940
494 16980
642 17020
827 17140
1156 17300
820 17420
506 17460
563 17660
903 17780
451 17820
648 17860
914 17900
921 17980
619 18020
846 18140
525 18580
652 18700
787 18880
1001 19060
618 19100
858 19140
682 19180
548 19300
919 19340
665 19380
786 19460
854 19500
583 19660
748 19740
722 19780
519 19820
899 20020


In [44]:
one_df = clean_df[clean_df['GeoFIPS']==10900]

one_df[[str(x) for x in range(1969,2001)]].isnull().sum()

1969    30
1970    29
1971    27
1972    30
1973    27
1974    25
1975    24
1976    25
1977    26
1978    23
1979    22
1980    21
1981    21
1982    23
1983    24
1984    28
1985    26
1986    26
1987    25
1988    26
1989    28
1990    26
1991    27
1992    31
1993    25
1994    26
1995    21
1996    25
1997    24
1998    26
1999    29
2000    32
dtype: int64

In [53]:
one_yr_df = one_df[['LineCode','IndustryClassification','Description','2000']]

n = 900
print one_yr_df[one_yr_df['LineCode'] == n]
one_yr_df[one_yr_df['LineCode'] > n][one_yr_df['LineCode'] < n+100]

     LineCode IndustryClassification  \
898       900                    ...   

                                   Description     2000  
898      Government and government enterprises  1754040  


,LineCode,IndustryClassification,Description,2000
899,910,...,"Federal, civilian",175402
900,920,...,Military,29030
901,930,...,State and local,1549608
902,931,...,State government,NaN
903,932,...,Local government,NaN


In [87]:
print one_yr_df[one_yr_df['LineCode'] < 100]

print 'industries sum:', one_yr_df[one_yr_df['LineCode']%100==0]['2004'].sum()


     LineCode IndustryClassification  \
917        10                    ...   
918        20                    ...   
919        30                    ...   
920        35                    ...   
921        36                    ...   
922        37                    ...   
923        38                    ...   
924        42                    ...   
925        45                    ...   
926        46                    ...   
927        47                    ...   
928        50                    ...   
929        60                    ...   
930        61                    ...   
931        62                    ...   
932        70                    ...   
933        71                    ...   
934        72                    ...   
935        81                111-112   
936        82                    ...   
937        90                113-814   

                                           Description      2004  
917                                    Personal inco